# Descargar los archivos
https://ops-scala.nyc3.digitaloceanspaces.com/SAT/20211031CHEQUES.7z

https://ops-scala.nyc3.digitaloceanspaces.com/SAT/202109DEUDORES.7Z





In [ ]:
import pandas
from pandas import DataFrame as df

In [ ]:
import requests

In [ ]:
with open("downloads/cheques.7z", "wb") as f:
  f.write(requests.get("https://ops-scala.nyc3.digitaloceanspaces.com/SAT/20211031CHEQUES.7z").content)

with open("downloads/deudores.7z", "wb") as f:
  f.write(requests.get("https://ops-scala.nyc3.digitaloceanspaces.com/SAT/202109DEUDORES.7Z").content)

In [ ]:

!py7zr x downloads/cheques.7z
!py7zr x downloads/deudores.7z

In [ ]:
!head -10 downloads/deudores.txt
!wc -l downloads/deudores.txt
# !shuf -n 1000000 downloads/deudores.txt > downloads/deudores2.txt

In [ ]:
float("01")

In [ ]:
with open("downloads/deudores.txt", "r") as f:
  line = f.readline()

def bcra_deudores(line):
    entity = line[:5].strip().zfill(5)
    year = int(line[5:9].strip())
    month = int(line[9:11].zfill(2))
    id_type = int(line[11:13].strip().zfill(2))
    cuit = line[13:24].strip()
    debt_status = int(line[27:29].strip())
    total_debt = float(line[29:41].strip().replace(',', '.'))
    return (entity, year, month, id_type, cuit, debt_status, total_debt)


bcra_deudores(line)

In [ ]:
deudores = pandas.read_fwf("downloads/deudores.txt",
                          colspecs=[(0, 5), (5, 9), (9, 11), (13, 24), (27, 29), (29, 41)],
                          names=["banco", "year", "month", "cuit", "situacion", "monto"]
)

In [ ]:
deudores.head()

In [1]:
import sqlite3
conn = sqlite3.connect('bcra.db')
cur = conn.cursor()

In [ ]:


sql = '''
CREATE TABLE deudores (
  banco VARCHAR(5),
  year INT,
  month INT,
  cuit VARCHAR(11),
  situacion INT,
  monto INT
)'''

cur.execute(sql)

In [2]:
cur.execute("SELECT * FROM deudores")
cur.fetchall()

[('7', 2021, 9, '20005807523', 1, '118,0'),
 ('7', 2021, 9, '20018144299', 1, '2,0'),
 ('7', 2021, 9, '20005656800', 1, '27,0'),
 ('7', 2021, 9, '20006602992', 1, '100,0'),
 ('7', 2021, 9, '20001531256', 1, '3,0'),
 ('7', 2021, 9, '20005887497', 1, '2,0'),
 ('7', 2021, 9, '20034578657', 4, '9,0'),
 ('7', 2021, 9, '20005676194', 1, '53,0'),
 ('7', 2021, 9, '20004166257', 1, '15,0'),
 ('7', 2021, 9, '20040325035', 1, '1,0'),
 ('7', 2021, 9, '20005848386', 1, '221,0'),
 ('7', 2021, 9, '20004224311', 1, '42,0'),
 ('7', 2021, 9, '20005757992', 1, '31,0'),
 ('7', 2021, 9, '20017364643', 1, '6,0'),
 ('7', 2021, 9, '20002669200', 1, '4,0'),
 ('7', 2021, 9, '20018152690', 1, '18,0'),
 ('7', 2021, 9, '20030273991', 1, '71,0'),
 ('7', 2021, 9, '20017359054', 1, '7,0'),
 ('7', 2021, 9, '20032462392', 2, '4,0'),
 ('7', 2021, 9, '20038803922', 1, '49,0'),
 ('7', 2021, 9, '20013129232', 1, '3,0'),
 ('7', 2021, 9, '20031791694', 1, '162,0'),
 ('7', 2021, 9, '20040664514', 1, '3,0'),
 ('7', 2021, 9, '2

In [ ]:
deudores.to_records(index=False)

In [ ]:
import numpy as np

In [ ]:
def f(x):
  return int(x)

sqlite3.register_adapter(np.int32, lambda x: int(x))
sqlite3.register_adapter(np.int64, lambda x: int(x))

In [ ]:
cur.executemany("INSERT INTO deudores VALUES (?, ?, ?, ?, ?, ?)", deudores.to_records(index=False))

In [ ]:
cur.execute("SELECT COUNT(*) FROM deudores").fetchall()

In [ ]:
cur.execute("CREATE INDEX deudores_cuit ON deudores (cuit)")
cur.execute("CREATE INDEX deudores_banco ON deudores (banco)")

In [ ]:
cur.execute("SELECT banco, SUM(monto) FROM deudores GROUP BY banco ORDER BY SUM(monto) DESC")
cur.fetchall()

In [ ]:
cur.execute("SELECT cuit, SUM(monto) FROM deudores GROUP BY cuit ORDER BY SUM(monto) DESC LIMIT 1")
cur.fetchall()

In [ ]:
cur.execute("SELECT cuit, SUM(monto) FROM deudores WHERE SUBSTR(cuit, 1, 1) = '2' GROUP BY cuit ORDER BY SUM(monto) DESC LIMIT 1")
cur.fetchall()

In [ ]:
cur.execute("SELECT * FROM deudores WHERE cuit = '20061455540'")
cur.fetchall()

In [ ]:
cur.execute("""
SELECT cuit, SUM(monto), MAX(situacion)
FROM deudores
WHERE SUBSTR(cuit, 1, 1) = '2' AND situacion > 1
GROUP BY cuit
ORDER BY SUM(monto) DESC LIMIT 1""")
cur.fetchall()

In [ ]:
situaciones = []
montos = []

for row in cur.execute("""
  SELECT CASE WHEN situacion = '11' THEN 1 ELSE situacion END, SUM(monto) AS total
  FROM deudores
  WHERE SUBSTR(cuit, 1, 1) = '2'
  GROUP BY CASE WHEN situacion = '11' THEN 1 ELSE situacion END
  ORDER BY situacion"""):
  situaciones.append(row[0])
  montos.append(row[1])

print(dict(zip(situaciones, montos)))

In [ ]:
for i, x in enumerate(situaciones):
  # Format strings
  print(f"{x}, {1000 * montos[i]:.0f}, {montos[i] / sum(montos):.2%}")

In [ ]:
for row in cur.execute("""
  SELECT d.banco, sit5 / SUM(monto) AS total
  FROM deudores AS d
  INNER JOIN (
    SELECT banco, SUM(monto) AS sit5
    FROM deudores
    WHERE situacion > 1 AND situacion < 10
      AND CAST(banco AS INT) < 10000
    GROUP BY banco
  ) AS s5 ON d.banco = s5.banco
  GROUP BY d.banco, sit5
  HAVING SUM(monto) > 10000
  ORDER BY sit5 / SUM(monto) DESC """):
  print(row)

In [ ]:
for row in cur.execute("""
  SELECT CASE
    WHEN SUBSTR(cuit, 1, 2) = '20' THEN 'M'
    WHEN SUBSTR(cuit, 1, 2) = '27' THEN 'F'
  ELSE NULL END, situacion, COUNT(cuit)
  FROM deudores
  GROUP BY 1, 2
"""):
  print(row)

In [ ]:
for row in cur.execute("""
  SELECT
    banco,
    SUM(monto)
  FROM deudores
  WHERE situacion > 1 AND situacion < 10
  GROUP BY 1
  ORDER BY 2 DESC
"""):
  print(row)

In [3]:
cur.execute(""" SELECT * FROM deudores WHERE cuit = '23441600399'
            """)
cur.fetchall()

[('29', 2021, 9, '23441600399', 1, '202,0')]

In [ ]:
#save the db
conn.commit()